### Task1: CLEANING THE DATA

In [68]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re
import seaborn as sns
import scipy as sp
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score
import sklearn.metrics as metrics

In [69]:
promo_data = pd.read_csv('Hackathon_DimPromotion_SAN_vShared.csv')

In [70]:
print(promo_data.shape)

(3431, 4)


In [71]:
print(promo_data.head)

<bound method NDFrame.head of       PromotionKey            PromoMechanic PromotionStartDate  \
0            312237          Percentage off          10/1/2021   
1            265157           Buy $X for Y%          2/25/2020   
2            270553           Buy $X for $Y          1/15/2020   
3            308082  X units for $Y (X = 1)          3/10/2021   
4            317323                     PWP          5/13/2022   
...             ...                     ...                ...   
3426         297974  X units for $Y (X = 1)           1/1/2022   
3427         339287           Buy $X for $Y         12/12/2021   
3428         263837  X units for $Y (X = 1)          2/27/2020   
3429         375529  X units for $Y (X > 1)         11/16/2022   
3430         352362  X units for $Y (X > 1)          1/18/2020   

     PromotionEndDate  
0           10/6/2021  
1           2/25/2020  
2           1/27/2020  
3           3/29/2021  
4            7/6/2022  
...               ...  
3426     

In [72]:
#understanding entries
promo_data.PromoMechanic.unique()

array(['Percentage off', 'Buy $X for Y%', 'Buy $X for $Y',
       'X units for $Y (X = 1)', 'PWP', 'X units for Y%',
       'X units for $Y (X > 1)', 'Member Points', 'Xth for Y%',
       'Coupon/Voucher', 'Buy X free X', 'GWP', 'Buy X free Y', 'Unknown',
       'Price off', 'Xth @ $Y'], dtype=object)

In [73]:
#understanding entries
trans_data = pd.read_csv('Hackathon_FactSalesTransactionDATES_vShared.csv')

print(trans_data.head)

In [75]:
type(trans_data["ActualSales"])

pandas.core.series.Series

In [76]:
print(trans_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 9 columns):
 #   Column           Non-Null Count    Dtype  
---  ------           --------------    -----  
 0   TransactionDate  1046320 non-null  object 
 1   DayOfWeek        1046320 non-null  object 
 2   WeekendFlag      1046320 non-null  object 
 3   StoreKey         1046320 non-null  object 
 4   ProductKey       1046320 non-null  object 
 5   UnitVolume       1046320 non-null  float64
 6   ActualSales      1039620 non-null  float64
 7   SalesDiscount    1046320 non-null  float64
 8   RetailFullPrice  1046320 non-null  float64
dtypes: float64(4), object(5)
memory usage: 72.0+ MB
None


In [77]:
trans_data.isna().any().sum()

9

In [78]:
trans_data = trans_data.dropna(how='all')

In [79]:
print(trans_data.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1046320 entries, 0 to 1048574
Data columns (total 9 columns):
 #   Column           Non-Null Count    Dtype  
---  ------           --------------    -----  
 0   TransactionDate  1046320 non-null  object 
 1   DayOfWeek        1046320 non-null  object 
 2   WeekendFlag      1046320 non-null  object 
 3   StoreKey         1046320 non-null  object 
 4   ProductKey       1046320 non-null  object 
 5   UnitVolume       1046320 non-null  float64
 6   ActualSales      1039620 non-null  float64
 7   SalesDiscount    1046320 non-null  float64
 8   RetailFullPrice  1046320 non-null  float64
dtypes: float64(4), object(5)
memory usage: 79.8+ MB
None


In [80]:
print(trans_data.head)

<bound method NDFrame.head of         TransactionDate  DayOfWeek WeekendFlag StoreKey ProductKey  \
0               29/1/20  Wednesday       False      118      49334   
1              31/10/21     Sunday        True       78      59511   
2               17/6/21   Thursday       False      672      97438   
3                6/2/22     Sunday        True      193      58921   
4               29/3/22    Tuesday       False     2059      44633   
...                 ...        ...         ...      ...        ...   
1048570        24/11/20    Tuesday       False       56      49328   
1048571        30/12/21   Thursday       False      894      49325   
1048572        28/10/22     Friday       False      597      49326   
1048573         23/1/21   Saturday        True      285      49328   
1048574        26/11/20   Thursday       False      378      44784   

         UnitVolume  ActualSales  SalesDiscount  RetailFullPrice  
0               2.0    103.35080      -16.87360         120.22

In [81]:
trans_data['ActualSales'] = trans_data.RetailFullPrice + trans_data.SalesDiscount

In [82]:
# Count NaN values in multiple columns of DataFrame
nan_count = trans_data.isna().sum()
print(nan_count)

TransactionDate    0
DayOfWeek          0
WeekendFlag        0
StoreKey           0
ProductKey         0
UnitVolume         0
ActualSales        0
SalesDiscount      0
RetailFullPrice    0
dtype: int64


print(promo_data.info())

In [83]:
nan_count_promo = promo_data.isna().sum()
print(nan_count_promo)

PromotionKey          0
PromoMechanic         0
PromotionStartDate    0
PromotionEndDate      0
dtype: int64


In [84]:
promo_data["PromotionStartDate"] = pd.to_datetime(promo_data["PromotionStartDate"])

In [85]:
promo_data["PromotionEndDate"] = pd.to_datetime(promo_data["PromotionEndDate"])

In [86]:
print(promo_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3431 entries, 0 to 3430
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   PromotionKey        3431 non-null   int64         
 1   PromoMechanic       3431 non-null   object        
 2   PromotionStartDate  3431 non-null   datetime64[ns]
 3   PromotionEndDate    3431 non-null   datetime64[ns]
dtypes: datetime64[ns](2), int64(1), object(1)
memory usage: 107.3+ KB
None


In [87]:
promo_data.rename(columns = {'PromotionKey ':'PromotionKey'}, inplace = True)

In [88]:
product_data = pd.read_csv('Hackathon_DimProduct_SAN_vShared.csv')

In [89]:
print(product_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 298 entries, 0 to 297
Data columns (total 5 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   ProductKey            298 non-null    object
 1   BrandKey              298 non-null    int64 
 2   SupplierKey           298 non-null    int64 
 3   ProductCategory_Lvl1  298 non-null    object
 4   ProductCategory_Lvl2  298 non-null    object
dtypes: int64(2), object(3)
memory usage: 11.8+ KB
None


In [90]:
product_data['ProductKey'] = product_data['ProductKey'].str[-5:]
print (product_data['ProductKey'])

0      52138
1      19864
2      95144
3      23070
4      93165
       ...  
293    12353
294    55442
295    78206
296    55438
297    96971
Name: ProductKey, Length: 298, dtype: object


In [91]:
product_data['ProductKey'] = pd.to_numeric(product_data['ProductKey'])

In [92]:
print(product_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 298 entries, 0 to 297
Data columns (total 5 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   ProductKey            298 non-null    int64 
 1   BrandKey              298 non-null    int64 
 2   SupplierKey           298 non-null    int64 
 3   ProductCategory_Lvl1  298 non-null    object
 4   ProductCategory_Lvl2  298 non-null    object
dtypes: int64(3), object(2)
memory usage: 11.8+ KB
None


In [93]:
store_data = pd.read_csv('Hackathon_DimStore_SAN_vShared.csv')

In [94]:
print(store_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 739 entries, 0 to 738
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   StoreKey             739 non-null    int64 
 1   DistributionChannel  739 non-null    object
 2   StoreType            739 non-null    object
 3   Region_Lvl1          739 non-null    object
 4   Region_Lvl2          739 non-null    object
dtypes: int64(1), object(4)
memory usage: 29.0+ KB
None


In [95]:
trans_promo_data = pd.read_csv('Hackathon_FactSalesTransactionPromotion_vShared.csv')

In [96]:
print(trans_promo_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5318761 entries, 0 to 5318760
Data columns (total 4 columns):
 #   Column           Dtype 
---  ------           ----- 
 0   TransactionDate  object
 1   StoreKey         object
 2   ProductKey       int64 
 3   PromotionKey     int64 
dtypes: int64(2), object(2)
memory usage: 162.3+ MB
None


In [97]:
trans_promo_data["TransactionDate"] = pd.to_datetime(trans_promo_data["TransactionDate"])

In [98]:
trans_promo_data["StoreKey"] = trans_promo_data["StoreKey"].str.replace('\D', '', regex=True)

In [99]:
trans_promo_data["StoreKey"] = pd.to_numeric(trans_promo_data["StoreKey"])

In [100]:
print(trans_promo_data)

        TransactionDate  StoreKey  ProductKey  PromotionKey
0            2021-06-13       615       49492        348444
1            2020-01-15       118       26841        324048
2            2021-01-06       653       97953        273352
3            2022-04-24       288       95792        356728
4            2022-08-28       382       62330        364487
...                 ...       ...         ...           ...
5318756      2020-04-09       358       41631        267707
5318757      2022-03-13       576       95836        270013
5318758      2021-05-08       358       59736        256313
5318759      2021-10-31       193       95822        326071
5318760      2021-10-14       621       88531        312322

[5318761 rows x 4 columns]


In [101]:
print(trans_promo_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5318761 entries, 0 to 5318760
Data columns (total 4 columns):
 #   Column           Dtype         
---  ------           -----         
 0   TransactionDate  datetime64[ns]
 1   StoreKey         int64         
 2   ProductKey       int64         
 3   PromotionKey     int64         
dtypes: datetime64[ns](1), int64(3)
memory usage: 162.3 MB
None


In [102]:
trans_promo_data_mech = pd.merge(trans_promo_data, promo_data, how = 'inner', on = 'PromotionKey')
trans_promo_data_mech

,TransactionDate,StoreKey,ProductKey,PromotionKey,PromoMechanic,PromotionStartDate,PromotionEndDate
0,2021-06-13,615,49492,348444,PWP,2021-05-05,2021-06-30
1,2021-06-17,586,26844,348444,PWP,2021-05-05,2021-06-30
2,2021-06-12,422,49492,348444,PWP,2021-05-05,2021-06-30
3,2021-06-30,449,49493,348444,PWP,2021-05-05,2021-06-30
4,2021-05-25,342,94310,348444,PWP,2021-05-05,2021-06-30
...,...,...,...,...,...,...,...
5318756,2021-05-29,896,49329,340753,Buy $X for $Y,2021-05-27,2021-06-08
5318757,2020-12-12,896,44784,324832,Buy $X for $Y,2020-12-12,2020-12-12
5318758,2020-12-12,896,49489,324832,Buy $X for $Y,2020-12-12,2020-12-12
5318759,2022-01-14,896,49331,335744,Buy $X for $Y,2022-01-14,2022-01-16


In [103]:
trans_data["TransactionDate"] = pd.to_datetime(trans_data["TransactionDate"])

In [104]:
print(trans_data.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1046320 entries, 0 to 1048574
Data columns (total 9 columns):
 #   Column           Non-Null Count    Dtype         
---  ------           --------------    -----         
 0   TransactionDate  1046320 non-null  datetime64[ns]
 1   DayOfWeek        1046320 non-null  object        
 2   WeekendFlag      1046320 non-null  object        
 3   StoreKey         1046320 non-null  object        
 4   ProductKey       1046320 non-null  object        
 5   UnitVolume       1046320 non-null  float64       
 6   ActualSales      1046320 non-null  float64       
 7   SalesDiscount    1046320 non-null  float64       
 8   RetailFullPrice  1046320 non-null  float64       
dtypes: datetime64[ns](1), float64(4), object(4)
memory usage: 79.8+ MB
None


In [105]:
trans_data["Trans_Sale_Amount"] = trans_data.ActualSales * trans_data.UnitVolume
trans_data

,TransactionDate,DayOfWeek,WeekendFlag,StoreKey,ProductKey,UnitVolume,ActualSales,SalesDiscount,RetailFullPrice,Trans_Sale_Amount
0,2020-01-29,Wednesday,False,118,49334,2.0,103.35080,-16.87360,120.2244,206.70160
1,2021-10-31,Sunday,True,78,59511,1.0,17.13725,-17.13725,34.2745,17.13725
2,2021-06-17,Thursday,False,672,97438,1.0,22.46298,-14.97532,37.4383,22.46298
3,2022-06-02,Sunday,True,193,58921,2.0,42.18400,-9.49140,51.6754,84.36800
4,2022-03-29,Tuesday,False,2059,44633,1.0,31.63800,-12.65520,44.2932,31.63800
...,...,...,...,...,...,...,...,...,...,...
1048570,2020-11-24,Tuesday,False,56,49328,1.0,32.69260,0.00000,32.6926,32.69260
1048571,2021-12-30,Thursday,False,894,49325,1.0,27.99963,-3.11107,31.1107,27.99963
1048572,2022-10-28,Friday,False,597,49326,7.0,239.92150,0.00000,239.9215,1679.45050
1048573,2021-01-23,Saturday,True,285,49328,2.0,65.38520,0.00000,65.3852,130.77040


In [106]:
sum_of_all_sales = trans_data['Trans_Sale_Amount'].sum()
print(sum_of_all_sales)

537504186.63364


In [107]:
sum_of_all_vol = trans_data['UnitVolume'].sum()
print(sum_of_all_vol)

2569573.0


In [108]:
sum_of_all_disc = (trans_data['SalesDiscount'].abs()* trans_data['UnitVolume']).sum()
print(sum_of_all_disc)

104056429.37982003


In [109]:
trans_promo_data_mech.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5318761 entries, 0 to 5318760
Data columns (total 7 columns):
 #   Column              Dtype         
---  ------              -----         
 0   TransactionDate     datetime64[ns]
 1   StoreKey            int64         
 2   ProductKey          int64         
 3   PromotionKey        int64         
 4   PromoMechanic       object        
 5   PromotionStartDate  datetime64[ns]
 6   PromotionEndDate    datetime64[ns]
dtypes: datetime64[ns](3), int64(3), object(1)
memory usage: 324.6+ MB


In [110]:
trans_data["StoreKey"] = trans_data["StoreKey"].str.replace('\D', '', regex=True)
trans_data["StoreKey"] = pd.to_numeric(trans_data["StoreKey"])
trans_data["ProductKey"] = trans_data["ProductKey"].str.replace('\D', '', regex=True)
trans_data["ProductKey"] = pd.to_numeric(trans_data["ProductKey"])
trans_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1046320 entries, 0 to 1048574
Data columns (total 10 columns):
 #   Column             Non-Null Count    Dtype         
---  ------             --------------    -----         
 0   TransactionDate    1046320 non-null  datetime64[ns]
 1   DayOfWeek          1046320 non-null  object        
 2   WeekendFlag        1046320 non-null  object        
 3   StoreKey           1046320 non-null  int64         
 4   ProductKey         1046320 non-null  int64         
 5   UnitVolume         1046320 non-null  float64       
 6   ActualSales        1046320 non-null  float64       
 7   SalesDiscount      1046320 non-null  float64       
 8   RetailFullPrice    1046320 non-null  float64       
 9   Trans_Sale_Amount  1046320 non-null  float64       
dtypes: datetime64[ns](1), float64(5), int64(2), object(2)
memory usage: 87.8+ MB


In [111]:
trans_data_promo = trans_data.merge(trans_promo_data_mech, on=['TransactionDate','ProductKey', 'StoreKey'], how='left')

In [112]:
trans_data_promo

,TransactionDate,DayOfWeek,WeekendFlag,StoreKey,ProductKey,UnitVolume,ActualSales,SalesDiscount,RetailFullPrice,Trans_Sale_Amount,PromotionKey,PromoMechanic,PromotionStartDate,PromotionEndDate
0,2020-01-29,Wednesday,False,118,49334,2.0,103.35080,-16.87360,120.2244,206.70160,306999.0,X units for $Y (X = 1),2019-12-19,2020-02-10
1,2021-10-31,Sunday,True,78,59511,1.0,17.13725,-17.13725,34.2745,17.13725,295634.0,X units for $Y (X = 1),2021-07-29,2021-12-31
2,2021-06-17,Thursday,False,672,97438,1.0,22.46298,-14.97532,37.4383,22.46298,321993.0,PWP,2021-05-05,2021-06-30
3,2022-06-02,Sunday,True,193,58921,2.0,42.18400,-9.49140,51.6754,84.36800,NaN,NaN,NaT,NaT
4,2022-03-29,Tuesday,False,2059,44633,1.0,31.63800,-12.65520,44.2932,31.63800,258679.0,X units for $Y (X = 1),2022-01-04,2022-12-31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1046315,2020-11-24,Tuesday,False,56,49328,1.0,32.69260,0.00000,32.6926,32.69260,NaN,NaN,NaT,NaT
1046316,2021-12-30,Thursday,False,894,49325,1.0,27.99963,-3.11107,31.1107,27.99963,331383.0,X units for $Y (X = 1),2021-11-17,2022-01-10
1046317,2022-10-28,Friday,False,597,49326,7.0,239.92150,0.00000,239.9215,1679.45050,NaN,NaN,NaT,NaT
1046318,2021-01-23,Saturday,True,285,49328,2.0,65.38520,0.00000,65.3852,130.77040,NaN,NaN,NaT,NaT


In [113]:
trans_data_promo = trans_data_promo.astype({"SalesDiscount":'int', "UnitVolume":'int'}) 
trans_data_promo.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1046320 entries, 0 to 1046319
Data columns (total 14 columns):
 #   Column              Non-Null Count    Dtype         
---  ------              --------------    -----         
 0   TransactionDate     1046320 non-null  datetime64[ns]
 1   DayOfWeek           1046320 non-null  object        
 2   WeekendFlag         1046320 non-null  object        
 3   StoreKey            1046320 non-null  int64         
 4   ProductKey          1046320 non-null  int64         
 5   UnitVolume          1046320 non-null  int64         
 6   ActualSales         1046320 non-null  float64       
 7   SalesDiscount       1046320 non-null  int64         
 8   RetailFullPrice     1046320 non-null  float64       
 9   Trans_Sale_Amount   1046320 non-null  float64       
 10  PromotionKey        403406 non-null   float64       
 11  PromoMechanic       403406 non-null   object        
 12  PromotionStartDate  403406 non-null   datetime64[ns]
 13  PromotionEnd

In [114]:
#sum_of_disc_df = trans_data_promo[['UnitVolume', 'SalesDiscount', 'PromoMechanic']].copy().dropna()
#sum_of_disc_df.info()

In [115]:
# sum_of_disc_PromoMech = {}
# for key, value in sum_of_disc_df.items():
#     PromoMech = sum_of_disc_df['PromoMechanic']
#     xDisc = sum_of_disc_df['SalesDiscount']*sum_of_disc_df['UnitVolume']
#     if PromoMech not in sum_of_disc_PromoMech:
#         sum_of_disc_PromoMech[PromoMech] = [xDisc]
#     else:
#         sum_of_disc_PromoMech[PromoMech].append(xDisc)
        
# sum_of_disc_PromoMech

In [116]:
sum_of_disc_df = trans_data_promo[['UnitVolume', 'SalesDiscount', 'PromoMechanic']].copy().dropna()
sum_of_disc_df['totdisc'] = (sum_of_disc_df['SalesDiscount'].abs())*sum_of_disc_df['UnitVolume']
sum_of_disc_df = sum_of_disc_df.drop(columns = ['UnitVolume', 'SalesDiscount'])
sum_of_disc_df.groupby(['PromoMechanic']).sum()

,totdisc
PromoMechanic,
Buy $X for $Y,12336168
Buy $X for Y%,566077
Buy X free X,3637021
PWP,21237123
Percentage off,3640119
Price off,2284
X units for $Y (X = 1),24714033
X units for $Y (X > 1),11397648
X units for Y%,1869043


In [117]:
store_data.info()
store_data.DistributionChannel.unique()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 739 entries, 0 to 738
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   StoreKey             739 non-null    int64 
 1   DistributionChannel  739 non-null    object
 2   StoreType            739 non-null    object
 3   Region_Lvl1          739 non-null    object
 4   Region_Lvl2          739 non-null    object
dtypes: int64(1), object(4)
memory usage: 29.0+ KB


array(['Physical', 'Online'], dtype=object)

In [118]:
trans_data_promo_store = trans_data_promo.merge(store_data, on=['StoreKey'], how='left')
trans_data_promo_store

,TransactionDate,DayOfWeek,WeekendFlag,StoreKey,ProductKey,UnitVolume,ActualSales,SalesDiscount,RetailFullPrice,Trans_Sale_Amount,PromotionKey,PromoMechanic,PromotionStartDate,PromotionEndDate,DistributionChannel,StoreType,Region_Lvl1,Region_Lvl2
0,2020-01-29,Wednesday,False,118,49334,2,103.35080,-16,120.2244,206.70160,306999.0,X units for $Y (X = 1),2019-12-19,2020-02-10,Physical,Store Type B,RegionB,Region 17
1,2021-10-31,Sunday,True,78,59511,1,17.13725,-17,34.2745,17.13725,295634.0,X units for $Y (X = 1),2021-07-29,2021-12-31,Physical,Store Type B,RegionA,Region 26
2,2021-06-17,Thursday,False,672,97438,1,22.46298,-14,37.4383,22.46298,321993.0,PWP,2021-05-05,2021-06-30,Physical,Store Type C,RegionB,Region 24
3,2022-06-02,Sunday,True,193,58921,2,42.18400,-9,51.6754,84.36800,NaN,NaN,NaT,NaT,Physical,Store Type B,RegionA,Region 31
4,2022-03-29,Tuesday,False,2059,44633,1,31.63800,-12,44.2932,31.63800,258679.0,X units for $Y (X = 1),2022-01-04,2022-12-31,Physical,Store Type B,RegionA,Region 203
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1064567,2020-11-24,Tuesday,False,56,49328,1,32.69260,0,32.6926,32.69260,NaN,NaN,NaT,NaT,Physical,Store Type D,RegionA,Region 57
1064568,2021-12-30,Thursday,False,894,49325,1,27.99963,-3,31.1107,27.99963,331383.0,X units for $Y (X = 1),2021-11-17,2022-01-10,Online,Store Type K,Online,Region 30
1064569,2022-10-28,Friday,False,597,49326,7,239.92150,0,239.9215,1679.45050,NaN,NaN,NaT,NaT,Physical,Store Type C,RegionA,Region 2
1064570,2021-01-23,Saturday,True,285,49328,2,65.38520,0,65.3852,130.77040,NaN,NaN,NaT,NaT,Physical,Store Type C,RegionB,Region 37


In [119]:
trans_data_promo_store['TotDisc'] = (trans_data_promo_store['SalesDiscount'].abs())*trans_data_promo_store['UnitVolume']
SumVolDisc_sales_stores = trans_data_promo_store[['Trans_Sale_Amount', 'DistributionChannel', 'UnitVolume', 'TotDisc']].groupby(['DistributionChannel']).sum()
SumVolDisc_sales_stores

,Trans_Sale_Amount,UnitVolume,TotDisc
DistributionChannel,,,
Online,1.296747e+08,74577,23340893
Physical,4.133712e+08,2535971,81024465


In [120]:
SumVolDisc_sales_stores = trans_data_promo_store[['Trans_Sale_Amount', 'StoreKey', 'UnitVolume', 'TotDisc']]
SumVolDisc_sales_store2071 = SumVolDisc_sales_stores[SumVolDisc_sales_stores['StoreKey'] == 2071].sum()
SumVolDisc_sales_store2071

Trans_Sale_Amount    1.783546e+05
StoreKey             2.033722e+06
UnitVolume           1.868000e+03
TotDisc              3.960200e+04
dtype: float64

In [121]:
product_data

,ProductKey,BrandKey,SupplierKey,ProductCategory_Lvl1,ProductCategory_Lvl2
0,52138,3521,1039040101,Category A,category aa
1,19864,3521,1039040125,Category A,Category AC
2,95144,1713,1049688101,Category A,Category AA
3,23070,3521,1050235101,Category A,Category AC
4,93165,3521,1039040125,Category A,Category AC
...,...,...,...,...,...
293,12353,3521,1036290101,Category A,Category AB
294,55442,132,1036290101,Category A,Category AC
295,78206,3521,1039040101,Category A,Category AC
296,55438,132,1036290101,Category A,Category AB


In [122]:
trans_data_promo_store_product = trans_data_promo_store.merge(product_data, on=['ProductKey'], how='left')
trans_data_promo_store_product
trans_data_promo_store_product.ProductCategory_Lvl2.unique()

array(['Category AC', 'Category AB', 'Category AA', 'Category AE',
       'Category AD', 'category ab', 'Categrory AE', 'Category  AC',
       'category aa', 'Category  AD'], dtype=object)

In [123]:
for i in trans_data_promo_store_product:
    # if trans_data_promo_store_product['ProductCategory_Lvl2'].str[-2:] == "AC":
    #     trans_data_promo_store_product['ProductCategory_Lvl2'] = "Category AC"
    trans_data_promo_store_product.loc[trans_data_promo_store_product["ProductCategory_Lvl2"].str[-2:] == "AC", "ProductCategory_Lvl2"] = "Category AC"

trans_data_promo_store_product
# trans_data_promo_store_product.ProductCategory_Lvl2.unique()

,TransactionDate,DayOfWeek,WeekendFlag,StoreKey,ProductKey,UnitVolume,ActualSales,SalesDiscount,RetailFullPrice,Trans_Sale_Amount,...,PromotionEndDate,DistributionChannel,StoreType,Region_Lvl1,Region_Lvl2,TotDisc,BrandKey,SupplierKey,ProductCategory_Lvl1,ProductCategory_Lvl2
0,2020-01-29,Wednesday,False,118,49334,2,103.35080,-16,120.2244,206.70160,...,2020-02-10,Physical,Store Type B,RegionB,Region 17,32,3521,1028407101,Category A,Category AC
1,2021-10-31,Sunday,True,78,59511,1,17.13725,-17,34.2745,17.13725,...,2021-12-31,Physical,Store Type B,RegionA,Region 26,17,3521,1039040101,Category A,Category AB
2,2021-06-17,Thursday,False,672,97438,1,22.46298,-14,37.4383,22.46298,...,2021-06-30,Physical,Store Type C,RegionB,Region 24,14,3521,1039040101,Category A,Category AC
3,2022-06-02,Sunday,True,193,58921,2,42.18400,-9,51.6754,84.36800,...,NaT,Physical,Store Type B,RegionA,Region 31,18,3521,1039040101,Category A,Category AA
4,2022-03-29,Tuesday,False,2059,44633,1,31.63800,-12,44.2932,31.63800,...,2022-12-31,Physical,Store Type B,RegionA,Region 203,12,3521,1039040101,Category A,Category AA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1064567,2020-11-24,Tuesday,False,56,49328,1,32.69260,0,32.6926,32.69260,...,NaT,Physical,Store Type D,RegionA,Region 57,0,3521,1028407101,Category A,Category AC
1064568,2021-12-30,Thursday,False,894,49325,1,27.99963,-3,31.1107,27.99963,...,2022-01-10,Online,Store Type K,Online,Region 30,3,3521,1039040118,Category A,Category AC
1064569,2022-10-28,Friday,False,597,49326,7,239.92150,0,239.9215,1679.45050,...,NaT,Physical,Store Type C,RegionA,Region 2,0,3521,1039040125,Category A,Category AC
1064570,2021-01-23,Saturday,True,285,49328,2,65.38520,0,65.3852,130.77040,...,NaT,Physical,Store Type C,RegionB,Region 37,0,3521,1028407101,Category A,Category AC


In [124]:
SumVolDisc_sales_prod = trans_data_promo_store_product[['Trans_Sale_Amount', 'ProductCategory_Lvl2', 'UnitVolume', 'TotDisc']]
SumVolDisc_sales_prodAC = SumVolDisc_sales_prod[SumVolDisc_sales_prod['ProductCategory_Lvl2'] == "Category AC"].sum()
SumVolDisc_sales_prodAC

Trans_Sale_Amount                                         261649653.65007
ProductCategory_Lvl2    Category ACCategory ACCategory ACCategory ACCa...
UnitVolume                                                         939643
TotDisc                                                          36506307
dtype: object

In [125]:
trans_data_promo_store_product.ProductKey.unique()

array([49334, 59511, 97438, 58921, 44633, 78564, 95144, 82315, 49489,
       49963, 49169, 49328, 95821, 78905, 26841, 49326, 78208, 32728,
       62330, 78906, 49341, 52107, 49333, 59736, 43547, 72598, 85818,
       95816, 44784, 49331, 88531, 44632, 49329, 49340, 72113, 78907,
       14412, 89922, 75906, 97953, 40863, 78206, 81294, 94311, 14529,
       49493, 88529, 52135, 95835, 78913, 88528, 49327, 43632, 87723,
       14397, 96762, 26844, 88527, 62008, 96971, 49325, 41632, 14435,
       49492, 49490, 97952, 40428, 52108, 35304, 29698, 14654, 49621,
       40432, 14608, 44786, 95822, 62010, 95792, 78912, 97439, 93290,
       94310, 39982, 93291, 98193, 62331, 54161, 49330, 89815, 89699,
       73606, 73605, 30553, 78207, 49332, 39983, 93165, 59999, 41631,
       59759, 27434, 89813, 79378, 78910, 72579, 86513, 50709, 78909,
       69898, 52530, 43435, 73604, 95818, 49495, 52138, 31088, 95836,
       73607, 93163, 78914, 28134, 30905, 72582, 78911, 52446, 79377,
       36484, 75386,

In [126]:
SumVolDisc_sales_prodK = trans_data_promo_store_product[['Trans_Sale_Amount', 'ProductKey', 'UnitVolume', 'TotDisc']]
SumVolDisc_sales_prodKey = SumVolDisc_sales_prodK[SumVolDisc_sales_prodK['ProductKey'] == 49489].sum()
SumVolDisc_sales_prodKey

Trans_Sale_Amount    2.021076e+06
ProductKey           5.428943e+08
UnitVolume           2.149200e+04
TotDisc              4.257110e+05
dtype: float64

In [127]:
SumVolDisc_sales_prodKey.to_csv('SumVolDisc_sales_prodKey.csv')
SumVolDisc_sales_prodAC.to_csv('SumVolDisc_sales_prodAC.csv')
trans_data_promo_store_product.to_csv('trans_data_promo_store_product.csv')
